In [24]:
import pandas as pd 
import numpy as np 
import mysql.connector 
import datetime 

In [2]:
mydb=mysql.connector.connect(
    host="localhost",
    user='Senthil',
    password='43560270',
    #database='employees'
)
print(mydb)

In [3]:
cursor=mydb.cursor()

In [ ]:
#sql = "CREATE DATABASE employee" 
#cursor.execute(sql) 

In [11]:
sql= "CREATE TABLE salary(empid int,firstname VARCHAR(100),lastname VARCHAR(100),email VARCHAR(50),phone VARCHAR(15),hire_date date,job_id VARCHAR(15),salary int)"


In [13]:
cursor.execute("USE employee")
cursor.execute(sql)

In [14]:
import csv 
import datetime 
filename="employees1.csv" 

In [16]:
with open(filename, 'r') as csvfile: 
    csvreader =csv.reader(csvfile) 
    a = next(csvreader) 
    for row in csvreader: 
        print(row) 
        empid = int(row[0]) 
        firstname = row[1] 
        lastname = row[2] 
        email = row[3] 
        phone = row[4] 
        hire_date = datetime.datetime.strptime(row[5], '%d-%b-%y').date() 
        job_id = row[6] 
        salary = int(row[7]) 
        #salary = int(row[7][:-3]) 
        sql = "INSERT INTO salary(empid,firstname,lastname,email,phone,hire_date,job_id,salary) VALUES(%s,%s,%s,%s,%s,%s,%s,%s)" 
        val=(empid,firstname,lastname,email,phone,hire_date,job_id,salary) 
        cursor.execute(sql,val)
mydb.commit()
print("Data inserted successfully!")

['198', 'Donald', 'OConnell', 'DOCONNEL', '650.507.9833', '21-Jun-07', 'SH_CLERK', '2600']
['199', 'Douglas', 'Grant', 'DGRANT', '650.507.9844', '13-Jan-08', 'SH_CLERK', '2600']
['200', 'Jennifer', 'Whalen', 'JWHALEN', '515.123.4444', '17-Sep-03', 'AD_ASST', '4400']
['201', 'Michael', 'Hartstein', 'MHARTSTE', '515.123.5555', '17-Feb-04', 'MK_MAN', '13000']
['202', 'Pat', 'Fay', 'PFAY', '603.123.6666', '17-Aug-05', 'MK_REP', '6000']
['203', 'Susan', 'Mavris', 'SMAVRIS', '515.123.7777', '07-Jun-02', 'HR_REP', '6500']
['204', 'Hermann', 'Baer', 'HBAER', '515.123.8888', '07-Jun-02', 'PR_REP', '10000']
['205', 'Shelley', 'Higgins', 'SHIGGINS', '515.123.8080', '07-Jun-02', 'AC_MGR', '12008']
['206', 'William', 'Gietz', 'WGIETZ', '515.123.8181', '07-Jun-02', 'AC_ACCOUNT', '8300']
['100', 'Steven', 'King', 'SKING', '515.123.4567', '17-Jun-03', 'AD_PRES', '24000']
['101', 'Neena', 'Kochhar', 'NKOCHHAR', '515.123.4568', '21-Sep-05', 'AD_VP', '17000']
['102', 'Lex', 'De Haan', 'LDEHAAN', '515.123

In [23]:
import logging 
# Set up basic logging 
logging.basicConfig( 
    filename='etl_log.txt', 
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s' 
) 

In [25]:
csv_file_path = 'employees1.csv' 
df = pd.read_csv(csv_file_path) 
print("Raw data loaded:") 
print(df.head()) 
print(df.columns.tolist()) 
logging.info("CSV loaded successfully.")

Raw data loaded:
   EMPLOYEE_ID FIRST_NAME  LAST_NAME     EMAIL  PHONE_NUMBER  HIRE_DATE  \
0          198     Donald   OConnell  DOCONNEL  650.507.9833  21-Jun-07   
1          199    Douglas      Grant    DGRANT  650.507.9844  13-Jan-08   
2          200   Jennifer     Whalen   JWHALEN  515.123.4444  17-Sep-03   
3          201    Michael  Hartstein  MHARTSTE  515.123.5555  17-Feb-04   
4          202        Pat        Fay      PFAY  603.123.6666  17-Aug-05   

     JOB_ID  SALARY  
0  SH_CLERK    2600  
1  SH_CLERK    2600  
2   AD_ASST    4400  
3    MK_MAN   13000  
4    MK_REP    6000  
['EMPLOYEE_ID', 'FIRST_NAME', 'LAST_NAME', 'EMAIL', 'PHONE_NUMBER', 'HIRE_DATE', 'JOB_ID', 'SALARY']


In [26]:
# Handle missing values 
df.fillna({ 
    'EMAIL': 'not_provided@example.com', 
    'PHONE_NUMBER': '0000000000',
    'HIRE_DATE': '01-Jan-00', 
    'SALARY': 0 
}, inplace=True) 


In [27]:
# Standardize column names (optional) 
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns] 


In [29]:
print(df.columns.tolist()) 
# Convert hire_date from 'dd-MMM-yy' to 'YYYY-MM-DD' 
df['hire_date'] = pd.to_datetime(df['hire_date'], format='%d-%b-%y', errors='coerce') 

['employee_id', 'first_name', 'last_name', 'email', 'phone_number', 'hire_date', 'job_id', 'salary']


In [30]:
# Replace invalid dates with a default 
df['hire_date'] = df['hire_date'].fillna(pd.to_datetime('2000-01-01')) 
# Replace non-numeric salaries with 0 
df['salary'] = pd.to_numeric(df['salary'], errors='coerce').fillna(0).astype(int) 
logging.info("Data cleaning completed.") 

In [32]:
mydb = mysql.connector.connect( 
    host="localhost", 
    user="Senthil", 
    password="43560270", 
    database="employee"  # <-- change this 
) 

In [33]:
cursor = mydb.cursor() 

In [34]:
cursor.execute(""" 
    CREATE TABLE IF NOT EXISTS salary_2 ( 
        empid INT, 
        firstname VARCHAR(50), 
        lastname VARCHAR(50), 
        email VARCHAR(100), 
        phone VARCHAR(20), 
        hire_date DATE, 
        job_id VARCHAR(20), 
        salary INT 
    ) 
""") 

In [36]:
# Insert each row into the table 
for index, row in df.iterrows():
     sql = """ 
        INSERT INTO salary_2 (empid, firstname, lastname, email, phone, 
hire_date, job_id, salary) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
            firstname=VALUES(firstname), 
            lastname=VALUES(lastname), 
            email=VALUES(email), 
            phone=VALUES(phone), 
            hire_date=VALUES(hire_date), 
            job_id=VALUES(job_id), 
            salary=VALUES(salary) 
    """
     values = ( 
        int(row['employee_id']), 
        row['first_name'], 
        row['last_name'], 
        row['email'], 
        row['phone_number'], 
        row['hire_date'].date(), 
        row['job_id'], 
        int(row['salary']) 
    ) 
cursor.execute(sql, values) 
mydb.commit() 
cursor.close() 
mydb.close() 
logging.error("Something went wrong", exc_info=True) 
print("ETL process completed successfully.") 

ETL process completed successfully.


In [40]:
import os 
print(os.getcwd()) 

c:\Users\senth\Placement Training


In [51]:
!jupyter nbconvert --to script "Code 3.ipynb"

[NbConvertApp] Converting notebook Code 3.ipynb to script
C:\Users\senth\AppData\Local\Programs\Python\Python310\share\jupyter\nbconvert\templates\python\index.py.j2:61: UserWarning: IPython is needed to transform IPython syntax to pure Python. Install ipython if you need this functionality.
[NbConvertApp] Writing 4211 bytes to Code 3.py
